In [0]:
# https://keras.io/
!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU, ReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 128
num_classes = 10
#epochs = 300
epochs = 50
l = 10
num_filter = 20

Using TensorFlow backend.


In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 110s 1us/step


In [0]:
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

Changes made in the following network to **replace LeakyReLU by ReLU activation and remove Dense layer.**

In [0]:
input = Input(shape=(img_height, img_width, channel,))

# Layer 1
layer1 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input)
layer1 = BatchNormalization(name='norm_1')(layer1)
layer1 = ReLU()(layer1)
layer1 = MaxPooling2D(pool_size=(2, 2))(layer1)
#output: 16, 16, 32

# Layer 2
layer2 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2', use_bias=False)(layer1)
layer2 = BatchNormalization(name='norm_2')(layer2)
layer2 = ReLU()(layer2)
layer2 = MaxPooling2D(pool_size=(2, 2))(layer2)
#output: 8, 8, 64


# Layer 3
layer3 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3', use_bias=False)(layer2)
layer3 = BatchNormalization(name='norm_3')(layer3)
layer3 = ReLU()(layer3)
#output: 8, 8, 128

# Layer 4
layer4 = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_4', use_bias=False)(layer3)
layer4 = BatchNormalization(name='norm_4')(layer4)
layer4 = ReLU()(layer4)
#output: 8, 8, 64

# Layer 5
layer5 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_5', use_bias=False)(layer4)
layer5 = BatchNormalization(name='norm_5')(layer5)
layer5 = ReLU()(layer5)
layer5 = MaxPooling2D(pool_size=(2, 2))(layer5)
#output: 4, 4, 128

# Layer 6
layer6 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_6', use_bias=False)(layer5)
layer6 = BatchNormalization(name='norm_6')(layer6)
layer6 = ReLU()(layer6)
#output: 4, 4, 256


# Layer 7
layer7 = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_7', use_bias=False)(layer6)
layer7 = BatchNormalization(name='norm_7')(layer7)
layer7 = ReLU()(layer7)
#output: 4, 4, 128

# Layer 8
layer8 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_8', use_bias=False)(layer7)
layer8 = BatchNormalization(name='norm_8')(layer8)
layer8 = ReLU()(layer8)
layer8 = MaxPooling2D(pool_size=(2, 2))(layer8)
#output: 2, 2, 256

# Layer 9
layer9 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_9', use_bias=False)(layer8)
layer9 = BatchNormalization(name='norm_9')(layer9)
layer9 = ReLU()(layer9)
#output: 2, 2, 512

# Layer 10
layer10 = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_10', use_bias=False)(layer9)
layer10 = BatchNormalization(name='norm_10')(layer10)
layer10 = ReLU()(layer10)
#output: 2, 2, 256


# Layer 11
layer11 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_11', use_bias=False)(layer10)
layer11 = BatchNormalization(name='norm_11')(layer11)
layer11 = ReLU()(layer11)
#output: 2, 2, 512

# Layer 12
layer12 = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_12', use_bias=False)(layer11)
layer12 = BatchNormalization(name='norm_12')(layer12)
layer12 = ReLU()(layer12)
#output: 2, 2, 256

skip_connection = layer12
#output: 2, 2, 256


# Layer 13
layer13 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_13', use_bias=False)(layer12)
layer13 = BatchNormalization(name='norm_13')(layer13)
layer13 = ReLU()(layer13)
#output: 2, 2, 512

layer13 = MaxPooling2D(pool_size=(2, 2))(layer13)
#output: 1, 1, 512


# Layer 14
layer14 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_14', use_bias=False)(layer13)
layer14 = BatchNormalization(name='norm_14')(layer14)
layer14 = ReLU()(layer14)
#output: 1, 1, 1024


# Layer 15
layer15 = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_15', use_bias=False)(layer14)
layer15 = BatchNormalization(name='norm_15')(layer15)
layer15 = ReLU()(layer15)
#output: 1, 1, 512

# Layer 16
layer16 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_16', use_bias=False)(layer15)
layer16 = BatchNormalization(name='norm_16')(layer16)
layer16 = ReLU()(layer16)
#output: 1, 1, 1024

# Layer 17
layer17 = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_17', use_bias=False)(layer16)
layer17 = BatchNormalization(name='norm_17')(layer17)
layer17 = ReLU()(layer17)
#output: 1, 1, 512

# Layer 18
layer18 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_18', use_bias=False)(layer17)
layer18 = BatchNormalization(name='norm_18')(layer18)
layer18 = ReLU()(layer18)
#output: 1, 1, 1024

# Layer 19
layer19 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_19', use_bias=False)(layer18)
layer19 = BatchNormalization(name='norm_19')(layer19)
layer19 = ReLU()(layer19)
#output: 1, 1, 1024


# Layer 20
layer20 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_20', use_bias=False)(layer19)
layer20 = BatchNormalization(name='norm_20')(layer20)
layer20 = ReLU()(layer20)
#output: 1, 1, 1024


# Layer 21
skip_connection = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_21', use_bias=False)(skip_connection)
skip_connection = BatchNormalization(name='norm_21')(skip_connection)
skip_connection = ReLU()(skip_connection)
#output:  2, 2, 64
skip_connection = Lambda(space_to_depth_x2)(skip_connection)
#output: 1, 1, 256


# Skip Layer 1             ######### Layer added to increase number of channels ###########
skip_connection = Conv2D(512, (1,1), strides=(1,1), padding='same', name='skip_conv_1', use_bias=False)(skip_connection)
skip_connection = BatchNormalization(name='skip_norm_1')(skip_connection)
skip_connection = ReLU()(skip_connection)
#output: 1, 1, 512

# Skip Layer 2             ######### Layer added to increase number of channels ###########
skip_connection = Conv2D(1024, (1,1), strides=(1,1), padding='same', name='skip_conv_2', use_bias=False)(skip_connection)
skip_connection = BatchNormalization(name='skip_norm_2')(skip_connection)
skip_connection = ReLU()(skip_connection)
#output: 1, 1, 1024


layer21 = concatenate([skip_connection, layer20])
#output: 1, 1, 2048

# Intermediate_layer_1        ######### 1x1 Layer added to decrease number of channels ###########
inter_layer1 = Conv2D(512, (1,1), strides=(1,1), padding='same', name='inter_conv_1', use_bias=False)(layer21)
inter_layer1 = BatchNormalization(name='inter_norm_1')(inter_layer1)
inter_layer1 = ReLU()(inter_layer1)
#output: 1, 1, 512

# Layer 22
#layer22 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_22', use_bias=False)(layer21)
layer22 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_22', use_bias=False)(inter_layer1)
layer22 = BatchNormalization(name='norm_22')(layer22)
layer22 = ReLU()(layer22)
# output: 1, 1, 1024

# Intermediate_layer_2         ######### 1x1 Layer added to decrease number of channels ###########
inter_layer2 = Conv2D(num_classes, (1,1), strides=(1,1), padding='same', name='inter_conv_2', use_bias=False)(layer22)
inter_layer2 = BatchNormalization(name='inter_norm_2')(inter_layer2)
inter_layer2 = ReLU()(inter_layer2)
# output: 1, 1, num_classes

# Layer 23
#layer23 = Flatten()(layer22)
layer23 = Flatten()(inter_layer2)


#output = Dense(num_classes, activation='softmax')(layer23)
output = Activation('softmax')(layer23)



In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 32, 32, 32)   864         input_5[0][0]                    
__________________________________________________________________________________________________
norm_1 (BatchNormalization)     (None, 32, 32, 32)   128         conv_1[0][0]                     
__________________________________________________________________________________________________
re_lu_53 (ReLU)                 (None, 32, 32, 32)   0           norm_1[0][0]                     
__________________________________________________________________________________________________
max_poolin

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 103s 2ms/step - loss: 1.5456 - acc: 0.4799 - val_loss: 2.2390 - val_acc: 0.3851
Epoch 2/50
50000/50000 [==============================] - 89s 2ms/step - loss: 1.1132 - acc: 0.6391 - val_loss: 1.3734 - val_acc: 0.5693
Epoch 3/50
50000/50000 [==============================] - 89s 2ms/step - loss: 0.8951 - acc: 0.7094 - val_loss: 1.1959 - val_acc: 0.6177
Epoch 4/50
50000/50000 [==============================] - 88s 2ms/step - loss: 0.7483 - acc: 0.7606 - val_loss: 1.0837 - val_acc: 0.6611
Epoch 5/50
50000/50000 [==============================] - 88s 2ms/step - loss: 0.6488 - acc: 0.7917 - val_loss: 0.9796 - val_acc: 0.6897
Epoch 6/50
50000/50000 [==============================] - 90s 2ms/step - loss: 0.5563 - acc: 0.8229 - val_loss: 1.2791 - val_acc: 0.6307
Epoch 7/50
50000/50000 [==============================] - 89s 2ms/step - loss: 

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0]) 
print('Test accuracy:', score[1])
# Save the trained weights in to .h5 format
model.save_weights("Yolo_Basic_model3.h5")
print("Saved the model to disk")

10000/10000 [==============================] - 9s 914us/step
Test loss: 0.9936264935970306
Test accuracy: 0.7834
Saved the model to disk


In [0]:
model.save_weights("Yolo_Basic_model5.h5")
print("Saved the model to disk")
from google.colab import files

files.download('Yolo_Basic_model5.h5')

Saved the model to disk
